# Zapytania złożone

Przed rozpoczęciem pracy konieczne jest skonfigurowanie bazy oraz załadowanie danych:
```ruby
$:.unshift "."
require 'db_setup'
require 'db_load'
```

In [1]:
$:.unshift "."
require 'db_setup'
require 'db_load'

==  AddIndicesMigration: migrating ============================================
==  AddIndicesMigration: migrated (0.0000s) ===================================



true

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
end
```

ActiveRecord udostępnia interfejs pozwalający wyszukiwać dane w bazie również z wykorzystaniem związków zdefiniowanych pomiędzy
klasami. Do definiowania tych związków służą polecenia `has_many`, `belongs_to`, `has_one` oraz `has_and_belongs_to_many`.
Przykładowo jeśli chcemy dowiedzieć się kto był autorem pierwszej książki wykonujemy zapytanie:
```ruby
book = Book.first
puts book.title
puts book.author
```

In [6]:
book = Book.first
puts book.title
puts book.author

Végjáték (regény)
#<Author:0x95cc82c>


Niestety prezentowany wynik nie jest zbyt użyteczny. Ponieważ jednak metoda `author` zwraca obiekt, możemy na nim wywoływać 
metody - tak samo jak w przypadku autora załadowanego bezpośrednio z wykorzystaniem metod zdefiniowanych dla klasy `Author`
```ruby
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"
```

In [8]:
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"

Végjáték (regény)
Orson Scott Card


Co więcej, jeśli mamy do czynienia ze związkiem jeden-do-wiele bądź wiele-do-wiele, możemy wywyływać wszystkie dotychczas poznane 
metody na kolekcji powiązanych obiektów. Istotnie ułatwia to wyszukiwanie potrzebnych danych:
```ruby
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end
```

In [9]:

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end

Anna Karenina


[#<Book id: 33, title: "Anna Karenina", language: nil, author_id: 31, published: 1877, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

Różnica pomiędzy tymi zapytaniami polega na tym, że w pierwszym przypadku szukamy książki, których tytuł zaczyna się na "A" 
w całym zbiorze książek, a w drugi przypadku wśród książek napisanych przez Tołstoja.

Aby umożliwić formatowanie wyników zapytania, IRuby udostępnia metodę `IRuby.html`, która pozwala na wyświetlanie kodu
HTML. Przykładowo
```ruby
IRuby.html "<h1>Test HTML-a</h1>"
```

In [10]:
IRuby.html "<h1>Test HTML-a</h1>"

"<h1>Test HTML-a</h1>"

Dzięki temu będziemy mogli formatować wyniki zapytań, a wszczególności wyświetlać obrazy przedstwiające autorów oraz książki. 
Należy jednak wziąć pod uwagę, że nie wszystkie książki oraz nie wszyscy autorzy posiadają odpowiadające im obrazki.
Aby wyświetlić imię, nazwisko oraz obraz pierwszego autora piszemy:
```ruby
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
```

In [11]:
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

Możemy również zdefniować funkcję, np. `display_author`, której zadaniem będzie wyświetlenie autora w tej postaci. 
Jedyny problem polega na tym, że funkcja `IRuby.html` może być wywoływana tylko raz. Dlatego jeśli chcemy wyświetlić kilku autorów,
musimy połączyć odpowiadające im kody HTML i wynik przekazać do funkcji `html`
```ruby
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document
```

In [12]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Andrzej Sapkowski</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/>"

## Zadania

### Zadanie 1

Wyświetl wszystkich autorów, którzy urodzili się przed rokiem 1900.

In [13]:
def display_authors(authors)
  html = ""
  authors.each do |author|
    html << "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
  end
  html
end
authors = []
authors = Author.where('born < ?',Date.parse("1900-01-01")).where("image_url is not null").all
document = display_authors(authors)
IRuby.html document

"<h1>Jack London</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300'/><h1>Mikołaj Kopernik</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300'/><h1>Maria Franciszka Kozłowska</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Mateczk_Kozłowska.JPG?width=300'/><h1>William Szekspir</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/><h1>John Ronald Reuel Tolkien</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><h1>Aleksander Dumas (ojciec)</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Dumas_by_Nadar,_1855.jpg?width=300'/><h1>Tadeusz Dołęga-Mostowicz</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tadeusz_Dolega-Mostowicz.jpg?width=300'/><h1>Vladimir Nabokov</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Monument_Nabokov_Montreux_23.12.2006.jpg?width=300'/><h1>Adolf Hitler</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hitler_portrait_crop.jpg?width=300'/><h1>Benedykt Chmielowski</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/POL_COA_Nałęcz.svg?width=300'/><h1>Aldous Huxley</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Aldous_Huxley.JPG?width=300'/><h1>Honoré de Balzac</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/HBalzac.jpg?width=300'/><h1>Franz Kafka</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kafka1906.jpg?width=300'/><h1>Howard Phillips Lovecraft</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Howard_Phillips_Lovecraft_in_1915.jpg?width=300'/><h1>Lew Tołstoj</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Ilya_Efimovich_Repin_(1844-1930)_-_Portrait_of_Leo_Tolstoy_(1887).jpg?width=300'/><h1>Ilja Ilf</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Ilya_Ilf.jpg?width=300'/><h1>Gabriela Zapolska</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Gabriela_Zapolska.PNG?width=300'/><h1>Clive Staples Lewis</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Statue_of_C.S._Lewis,_Belfast.jpg?width=300'/><h1>Iwan Turgieniew</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Turgenev_by_Repin.jpg?width=300'/><h1>Lucy Maud Montgomery</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Lucy_Maud_Montgomery.JPG?width=300'/><h1>Aleksander Majkowski</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/POL_Polonia_Restituta_Oficerski_BAR.svg?width=300'/>"

### Zadanie 2

Zmodyfikuj funkcję wyświetlającą autorów tak aby obejmowała ich datę urodzin i śmierci w formacie `dzień.miesiąc.rok`. Następnie wyświetl wszystkich autorów,
którzy zmarli w XX wieku.

In [14]:
def display_authors(authors)
  html = ""
  authors.each do |author|
    html << "<h1>#{author.name} #{author.surname} #{author.born.strftime("%m.%d.%Y")} - #{author.died.strftime("%m.%d.%Y")} </h1><img src='#{author.image_url}'/>"
end
  html
end
authors = []
authors = Author.where('born < ?',Date.parse("1900-01-01")).where("image_url is not null").where(died: Date.parse("1900-01-01")..Date.parse("2000-12-31")).all
document = display_authors(authors)
IRuby.html document

"<h1>Jack London 01.12.1876 - 11.22.1916 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300'/><h1>John Ronald Reuel Tolkien 01.03.1892 - 09.02.1973 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><h1>Tadeusz Dołęga-Mostowicz 08.10.1898 - 09.20.1939 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tadeusz_Dolega-Mostowicz.jpg?width=300'/><h1>Vladimir Nabokov 04.22.1899 - 07.02.1977 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Monument_Nabokov_Montreux_23.12.2006.jpg?width=300'/><h1>Adolf Hitler 04.20.1889 - 04.30.1945 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hitler_portrait_crop.jpg?width=300'/><h1>Aldous Huxley 07.26.1894 - 11.22.1963 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Aldous_Huxley.JPG?width=300'/><h1>Franz Kafka 07.03.1883 - 06.03.1924 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kafka1906.jpg?width=300'/><h1>Howard Phillips Lovecraft 08.20.1890 - 03.15.1937 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Howard_Phillips_Lovecraft_in_1915.jpg?width=300'/><h1>Lew Tołstoj 09.09.1828 - 11.20.1910 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Ilya_Efimovich_Repin_(1844-1930)_-_Portrait_of_Leo_Tolstoy_(1887).jpg?width=300'/><h1>Ilja Ilf 10.15.1897 - 04.13.1937 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Ilya_Ilf.jpg?width=300'/><h1>Gabriela Zapolska 03.30.1857 - 12.17.1921 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Gabriela_Zapolska.PNG?width=300'/><h1>Clive Staples Lewis 11.29.1898 - 11.22.1963 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Statue_of_C.S._Lewis,_Belfast.jpg?width=300'/><h1>Lucy Maud Montgomery 11.30.1874 - 04.24.1942 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Lucy_Maud_Montgomery.JPG?width=300'/><h1>Aleksander Majkowski 07.17.1876 - 02.10.1938 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/POL_Polonia_Restituta_Oficerski_BAR.svg?width=300'/>"

### Zadanie 3

Wyświetl 5 najmłodszych autorów, którzy posiadają przypisane zdjęcie.

In [15]:
def display_authors(authors)
  html = ""
  authors.each do |author|
    html << "<h1>#{author.name} #{author.surname} #{author.born} #{author.died}</h1><img src='#{author.image_url}'/>"
  end
  html
end
authors = []
authors = Author.order('born DESC').where("image_url is not null").limit(5)
document = display_authors(authors)
IRuby.html document

"<h1>Andrzej Pilipiuk 1974-03-20 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Andrzej_Pilipiuk-Polcon2006.jpg?width=300'/><h1>J. K. Rowling 1965-07-31 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/><h1>Rafał A. Ziemkiewicz 1964-09-13 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Rafał_Ziemkiewicz.jpg?width=300'/><h1>Boris Akunin 1956-05-20 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Boris_Akunin_2012.jpg?width=300'/><h1>Orson Scott Card 1951-08-24 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

### Zadanie 4

Zdefiniuj funkcję `display_book`, która będzie zwracała fragment kodu HTML obejmujący tytuł książki, rok jej wydania oraz
przypisany obrazek.
Wyświetl najwcześniej wydaną książkę.

In [16]:
def display_book(book)
  html = ""
  html << "<h1>#{book.title} #{book.published}</h1><img src='#{book.author.image_url}'/>"
  html
end

book = Book.order(:published).where("image_url is not null").first
document = display_book(book)
IRuby.html document

"<h1>Hamlet </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/>"

In [ ]:
IRuby.html "<h1>Test HTML-a</h1>"

### Zadanie 5

Wyświetl wszystkie książki, które nie mają przypisanego roku wydania.

In [17]:
def display_book(books)
  html = ""
  books.each do |book|
    html << "<h1>#{book.title} </h1><img src='#{book.author.image_url}'/>"
  end
  html
end
books = []
books = Book.where('published is null').where("image_url is not null").all
document = display_book(books)
IRuby.html document

"<h1>Hamlet </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/>"

### Zadanie 6

Zmodyfikuj funkcję `display_author` tak, aby wyświetlała również wszystkie książki napisane przez autora. Użyj znacznika `<ul>`.
Korzystając z tej funkcji wyświetl opis Stanisława Lema.

In [18]:
def display_author(surname)
  author = Author.where(surname: surname).first
  html = ""
  html << "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  author.books.each do |book|
    html << "<li> #{book.title} </li>"
  end
  html << "</ul>"
  html
end


document = display_author("Lem")
IRuby.html document

"<h1>Stanisław Lem</h1><ul><li> Bajki robotów </li><li> Solaris (powieść) </li><li> Bezsenność (zbiór opowiadań) </li><li> Głos Pana </li><li> Inwazja z Aldebarana (zbiór opowiadań) </li><li> Katar (powieść) </li><li> Księga robotów </li><li> Maska (zbiór dzieł) </li><li> Niezwyciężony i inne opowiadania </li><li> Noc księżycowa </li><li> Pamiętnik znaleziony w wannie </li><li> Polowanie (zbiór opowiadań) </li><li> Powtórka (zbiór dzieł) </li><li> Pożytek ze smoka </li><li> Rasa drapieżców. Teksty ostatnie </li><li> Sezam i inne opowiadania </li><li> Summa technologiae </li><li> Wizja lokalna (powieść) </li></ul>"

### Zadanie 7

Wśród książek napisanych przez Lema wyświetl wszystkie, które zawieraja w tytule słowo "zbiór".

In [19]:
def display_author(surname)
  author = Author.where(surname: surname).first
  html = ""
  html << "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  author.books.each do |book|
    html << "<li> #{book.title} </li>" if book.title =~ /zbiór/
  end
  html << "</ul>"
  html
end


document = display_author("Lem")
IRuby.html document

"<h1>Stanisław Lem</h1><ul><li> Bezsenność (zbiór opowiadań) </li><li> Inwazja z Aldebarana (zbiór opowiadań) </li><li> Maska (zbiór dzieł) </li><li> Polowanie (zbiór opowiadań) </li><li> Powtórka (zbiór dzieł) </li></ul>"

### Zadanie 8

Wśród książek napisanych przez Lema wyświetl te, które zostały wydane w roku jego śmierci.

In [31]:
def display_author(surname)
  author = Author.where(surname: surname).first
  html = ""
  html << "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  author.books.where(published: author.died.year).each do |book|
    html << "<li> #{book.title} </li>"  
  end
  html << "</ul>"
  html
end

document = display_author("Lem")
IRuby.html document

"<h1>Stanisław Lem</h1><ul><li> Rasa drapieżców. Teksty ostatnie </li></ul>"

### Zadanie 9

Wyświetl wszystkie książki znajdujące się w bazie. Opis powinien zawierać tytuł, imię i nazwisko autora, rok wydania oraz 
obraz przedstawiający książkę (o ile istnieje).

In [34]:
def display_book(books)
  html = ""
  books.each do |book|
    html << "<h1>#{book.title} #{book.author.name} #{book.author.surname} #{book.published}</h1><img src='#{book.author.image_url}'/>" 
  end
  html
end

books = Book.all
document = display_book(books)
IRuby.html document

"<h1>Végjáték (regény) Orson Scott Card </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Pani Jeziora (powieść) Andrzej Sapkowski 1999</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/><h1>Bellew Zawierucha Jack London 1912</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300'/><h1>Fałszywy trop (powieść) Henning Mankell 1995</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Henning_Mankell_3_2011_Shankbone.jpg?width=300'/><h1>Bajki robotów Stanisław Lem 1964</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><h1>Bolesław Chrobry (powieść) Antoni Gołubiew 1947</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Antoni_Golubiew.jpg?width=300'/><h1>De revolutionibus orbium coelestium Mikołaj Kopernik 1543</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300'/><h1>Dzieło Wielkiego Miłosierdzia Maria Franciszka Kozłowska 1922</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Mateczk_Kozłowska.JPG?width=300'/><h1>Folwark zwierzęcy George Orwell 1945</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/GeoreOrwell.jpg?width=300'/><h1>Hamlet William Szekspir </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/><h1>Harry Potter J. K. Rowling 1997</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/><h1>Hobbit, czyli tam i z powrotem John Ronald Reuel Tolkien 1937</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><h1>Hrabia Monte Christo Aleksander Dumas (ojciec) 1844</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Dumas_by_Nadar,_1855.jpg?width=300'/><h1>Kariera Nikodema Dyzmy Tadeusz Dołęga-Mostowicz 1932</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tadeusz_Dolega-Mostowicz.jpg?width=300'/><h1>Krzyżacy (powieść) Henryk Sienkiewicz 1900</h1><img src=''/><h1>Lolita Vladimir Nabokov 1955</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Monument_Nabokov_Montreux_23.12.2006.jpg?width=300'/><h1>Mein Kampf Adolf Hitler 1925</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hitler_portrait_crop.jpg?width=300'/><h1>Nad Niemnem Eliza Orzeszkowa 1888</h1><img src=''/><h1>Niedokończone opowieści Christopher Tolkien 1980</h1><img src=''/><h1>Nowe Ateny Benedykt Chmielowski 1745</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/POL_COA_Nałęcz.svg?width=300'/><h1>Nowy wspaniały świat Aldous Huxley 1932</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Aldous_Huxley.JPG?width=300'/><h1>Ogniem i mieczem Henryk Sienkiewicz 1884</h1><img src=''/><h1>Ojciec Goriot Honoré de Balzac 1835</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/HBalzac.jpg?width=300'/><h1>Proces (powieść) Franz Kafka 1925</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kafka1906.jpg?width=300'/><h1>Psałterz Dawidów Jan Kochanowski 1579</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Jan_Kochanowski.png?width=300'/><h1>Quo vadis Henryk Sienkiewicz 1896</h1><img src=''/><h1>Romeo i Julia William Szekspir 1597</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/><h1>Silmarillion John Ronald Reuel Tolkien 1977</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><h1>Solaris (powieść) Stanisław Lem 1961</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><h1>Władca Pierścieni John Ronald Reuel Tolkien 1954</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><h1>Zew Cthulhu (opowiadanie) Howard Phillips Lovecraft 1928</h1><img src='http:

### Zadanie 10

Wyświetl wszystkich autorów znajdujących się w bazie wraz z opisem zawierającym ich zdjęcie, datę urodzin i śmierci oraz spis 
opublikowanych książek. Pamiętaj o odpowiednim sformatowaniu dat oraz o tym, że nie wszyscy autorzy mają przypisane wszystkie
informacje.

In [40]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1>
   <h3>#{author.born} #{author.died}</h3>
   <img src='#{author.image_url}'/>" + display_books(author)
end

def display_authors(authors)
  zmienna = ""
  authors.each do |author|
    zmienna << display_author(author)
  end
  return zmienna
end

def display_books(author)
  zmienna_2 = ""
  author.books.each do |book|
    zmienna_2 << "<ul><li><h2>#{book.title}</h2> <h3>#{book.published}</h3></li></ul>"
  end
  return zmienna_2
end

authors = Author.all

document = display_authors(authors)
IRuby.html document

"<h1>Adam Mickiewicz</h1>\n   <h3>1798-12-24 1855-11-26</h3>\n   <img src=''/><ul><li><h2>Księgi narodu polskiego i pielgrzymstwa polskiego</h2> <h3>1832</h3></li></ul><h1>Juliusz Słowacki</h1>\n   <h3>1809-09-04 1849-04-03</h3>\n   <img src=''/><h1>Henryk Sienkiewicz</h1>\n   <h3>1846-05-05 1916-11-15</h3>\n   <img src=''/><ul><li><h2>Krzyżacy (powieść)</h2> <h3>1900</h3></li></ul><ul><li><h2>Ogniem i mieczem</h2> <h3>1884</h3></li></ul><ul><li><h2>Quo vadis</h2> <h3>1896</h3></li></ul><h1>Eliza Orzeszkowa</h1>\n   <h3>1841-06-06 1910-05-18</h3>\n   <img src=''/><ul><li><h2>Nad Niemnem</h2> <h3>1888</h3></li></ul><h1>Orson Scott Card</h1>\n   <h3>1951-08-24 </h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><ul><li><h2>Végjáték (regény)</h2> <h3></h3></li></ul><h1>Andrzej Sapkowski</h1>\n   <h3>1948-06-21 </h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/><ul><li><h2>Pani Jeziora (powieść)</h2> <h3>1999</h3></li></ul><h1>Jack London</h1>\n   <h3>1876-01-12 1916-11-22</h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300'/><ul><li><h2>Bellew Zawierucha</h2> <h3>1912</h3></li></ul><h1>Henning Mankell</h1>\n   <h3>1948-02-03 </h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Henning_Mankell_3_2011_Shankbone.jpg?width=300'/><ul><li><h2>Fałszywy trop (powieść)</h2> <h3>1995</h3></li></ul><h1>Stanisław Lem</h1>\n   <h3>1921-09-12 2006-03-27</h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><ul><li><h2>Bajki robotów</h2> <h3>1964</h3></li></ul><ul><li><h2>Solaris (powieść)</h2> <h3>1961</h3></li></ul><ul><li><h2>Bezsenność (zbiór opowiadań)</h2> <h3>1971</h3></li></ul><ul><li><h2>Głos Pana</h2> <h3>1968</h3></li></ul><ul><li><h2>Inwazja z Aldebarana (zbiór opowiadań)</h2> <h3>1959</h3></li></ul><ul><li><h2>Katar (powieść)</h2> <h3>1976</h3></li></ul><ul><li><h2>Księga robotów</h2> <h3>1961</h3></li></ul><ul><li><h2>Maska (zbiór dzieł)</h2> <h3>1976</h3></li></ul><ul><li><h2>Niezwyciężony i inne opowiadania</h2> <h3>1964</h3></li></ul><ul><li><h2>Noc księżycowa</h2> <h3>1963</h3></li></ul><ul><li><h2>Pamiętnik znaleziony w wannie</h2> <h3>1961</h3></li></ul><ul><li><h2>Polowanie (zbiór opowiadań)</h2> <h3>1965</h3></li></ul><ul><li><h2>Powtórka (zbiór dzieł)</h2> <h3>1979</h3></li></ul><ul><li><h2>Pożytek ze smoka</h2> <h3>1993</h3></li></ul><ul><li><h2>Rasa drapieżców. Teksty ostatnie</h2> <h3>2006</h3></li></ul><ul><li><h2>Sezam i inne opowiadania</h2> <h3>1954</h3></li></ul><ul><li><h2>Summa technologiae</h2> <h3>1964</h3></li></ul><ul><li><h2>Wizja lokalna (powieść)</h2> <h3>1982</h3></li></ul><h1>Antoni Gołubiew</h1>\n   <h3>1907-02-25 1979-06-27</h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Antoni_Golubiew.jpg?width=300'/><ul><li><h2>Bolesław Chrobry (powieść)</h2> <h3>1947</h3></li></ul><h1>Mikołaj Kopernik</h1>\n   <h3>1473-02-19 1543-05-24</h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300'/><ul><li><h2>De revolutionibus orbium coelestium</h2> <h3>1543</h3></li></ul><h1>Maria Franciszka Kozłowska</h1>\n   <h3>1862-05-27 </h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Mateczk_Kozłowska.JPG?width=300'/><ul><li><h2>Dzieło Wielkiego Miłosierdzia</h2> <h3>1922</h3></li></ul><h1>George Orwell</h1>\n   <h3>1903-06-25 1950-01-21</h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/GeoreOrwell.jpg?width=300'/><ul><li><h2>Folwark zwierzęcy</h2> <h3>1945</h3></li></ul><ul><li><h2>Droga na molo w Wigan</h2> <h3>1937</h3></li></ul><h1>William Szekspir</h1>\n   <h3>1564-04-23 1616-05-03</h3>\n   <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/><ul><li><h2>Hamlet</h2> <h3></h3></li></ul><ul><li><h2>Romeo i Julia</h2> <h3>1597</h3></li></ul><